In [1]:
import pandas as pd 
from sklearn.feature_selection import VarianceThreshold , SelectKBest , f_classif
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
import math
import itertools
from amogel.utils.ac import generate_ac_feature_selection
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# data preparation 
# download data from google drive 
!gdown --id 1tXtQwLBqoI6sjV6QTXR1Dbw_6MTexjez

# unzip the data and delete the zip file
!unzip -q xena_ggi_ppi_co.zip

In [2]:
# load omic data 
def load_omic_data(omic_file):
    omic_data = pd.read_csv(omic_file, sep="\t", index_col=0)
    return omic_data


def filtering_variance(df: pd.DataFrame , threshold:float): 
        
    default_shape = df.shape
    
    sel = VarianceThreshold(threshold=threshold)
    df_filtered  = pd.DataFrame(sel.fit_transform(df) , index=df.index , columns=df.columns[sel.get_support()])
    
    filter_shape = df_filtered.shape
    print(f"Variance filtering | Default shape: {default_shape} | Filtered shape: {filter_shape}")
    
    return df_filtered

def annovaf_filtering( df: pd.DataFrame , df_label: pd.DataFrame , target: str , threshold: float)->pd.DataFrame:
        
        assert isinstance(df , pd.DataFrame) , "Invalid input"
        assert isinstance(df_label , pd.DataFrame) , "Invalid input"
        assert target in df_label.columns , f"Invalid target column name, not found in the label dataframe | {df_label.columns.tolist()}"
        
        default_shape = df.shape
        
        df_merged = pd.concat([df , df_label[target]] , axis=1)
        df_merged_mean = df_merged.groupby(target).mean()
        combination = list(itertools.permutations(df_merged_mean.index , 2))
        selected_genes = []
        for idx , gene in enumerate(df_merged.columns[:-1]): 
            # calculate fold-change for each gene to different groups
            # generate possible groups 
            try:
                fold_change = [
                    math.log2(df_merged_mean.loc[group1,gene]/df_merged_mean.loc[group2 , gene]) for group1 , group2 in combination
                ]
                
                # check if the fold change is greater than the threshold
                if any(x > 1 for x in fold_change):
                    selected_genes.append(gene)
            except Exception as e:
                pass
                #selected_genes.append(gene)
            
        print(f"Selected genes: {len(selected_genes)} , {selected_genes[:5]}...")
        if len(selected_genes) > 10:
            df = df[selected_genes]
            
        sel = SelectKBest(score_func=f_classif, k=threshold)
        df_filtered = pd.DataFrame(sel.fit_transform(df, df_label[target]), index=df.index , columns=df.columns[sel.get_support()])
        
        filter_shape = df_filtered.shape
        print(f"ANOVA-F filtering | Default shape: {default_shape} | Filtered shape: {filter_shape}")
        
        return  df_filtered

In [3]:
# load omic
cnv = load_omic_data("cnv.txt")
cnv.index = cnv.index.str.split("|").str[0]
cnv.columns = ["-".join(col.split("-")[0:3]) for col in cnv.columns]
cnv = cnv.T
cnv.index.name = "sample"

mRNA = load_omic_data("mRNA.txt")
mRNA.columns = ["-".join(col.split("-")[0:3]) for col in mRNA.columns]
mRNA = mRNA.T
mRNA.index.name = "sample"

In [4]:
# load label 
label  = pd.read_csv("TCGAbiolinks.csv")
label = label[["patient","BRCA_Subtype_PAM50"]]
label.set_index("patient", inplace=True)
label.columns = ["class"]
label.index.name = "sample"

In [5]:
# check missing value 
print("cnv missing value: ", cnv.isnull().sum().sum())
print("mRNA missing value: ", mRNA.isnull().sum().sum())
print("label missing value: ", label.isnull().sum().sum())

label = label.dropna()

cnv missing value:  0
mRNA missing value:  0
label missing value:  2


In [6]:
common_samples = set(cnv.index) & set(mRNA.index) & set(label.index)

# filter data
cnv = cnv.loc[list(common_samples)]
mRNA = mRNA.loc[list(common_samples)]
label = label.loc[list(common_samples)]

In [7]:
# merge duplicated index with mean 
mRNA = mRNA.groupby(mRNA.index).mean()
mRNA.shape

(1066, 20530)

In [8]:
# sort index
cnv.sort_index(inplace=True)
mRNA.sort_index(inplace=True)
label.sort_index(inplace=True)

In [9]:
label_enc = OrdinalEncoder()
label["class"] = label_enc.fit_transform(label[["class"]])
label_enc.categories_

[array(['Basal', 'Her2', 'LumA', 'LumB', 'Normal'], dtype=object)]

In [10]:
label

,class
sample,
TCGA-3C-AAAU,2.0
TCGA-3C-AALI,1.0
TCGA-3C-AALJ,3.0
TCGA-3C-AALK,2.0
TCGA-4H-AAAK,2.0
...,...
TCGA-WT-AB44,2.0
TCGA-XX-A899,2.0
TCGA-XX-A89A,2.0


In [11]:
# load STRING database
string_link = pd.read_csv("9606.protein.links.detailed.v12.0.txt", sep="\s", engine="python")
string_info = pd.read_csv("9606.protein.info.v12.0.txt", sep="\t")

In [12]:
merged = pd.merge(string_link , string_info[["#string_protein_id", "preferred_name"]], left_on="protein1", right_on="#string_protein_id")
merged.rename(columns={"preferred_name":"protein1_name"}, inplace=True)
merged = pd.merge(merged , string_info[["#string_protein_id", "preferred_name"]], left_on="protein2", right_on="#string_protein_id")
merged.rename(columns={"preferred_name":"protein2_name"}, inplace=True)
merged = merged[["protein1_name", "protein2_name", "neighborhood" , "coexpression"]]


In [13]:
GGI = pd.read_csv("ggi.txt", sep="\t")

In [14]:
GGI.shape

(1243032, 37)

In [15]:
cnv_filter = filtering_variance(cnv, 0.01)
mrna_filter = filtering_variance(mRNA, 0.01)
cnv_filter = annovaf_filtering(cnv_filter, label, "class", 1000)
mrna_filter = annovaf_filtering(mrna_filter, label, "class", 1000)


Variance filtering | Default shape: (1066, 24776) | Filtered shape: (1066, 24776)
Variance filtering | Default shape: (1066, 20530) | Filtered shape: (1066, 19856)
Selected genes: 10075 , ['ACAP3', 'ACTRT2', 'AGRN', 'ANKRD65', 'ATAD3A']...
ANOVA-F filtering | Default shape: (1066, 24776) | Filtered shape: (1066, 1000)
Selected genes: 2991 , ['HIF3A', 'RNF17', 'HMGCLL1', 'LRRTM1', 'CAMKV']...
ANOVA-F filtering | Default shape: (1066, 19856) | Filtered shape: (1066, 1000)


In [16]:
cnv_filter = cnv_filter / cnv_filter.max().max()
mrna_filter = mrna_filter / mrna_filter.max().max()

In [17]:
X_train, X_test, _ , _ = train_test_split(label.index, label["class"], test_size=0.3, random_state=41)

In [18]:
cnv_train = cnv_filter.loc[X_train]
cnv_test = cnv_filter.loc[X_test]
mrna_train = mrna_filter.loc[X_train]
mrna_test = mrna_filter.loc[X_test]
label_train = label.loc[X_train]
label_test = label.loc[X_test]

In [19]:
from amogel.utils.ac import generate_ac_feature_selection
# merged cnv_train and mrna_train
train_data = pd.concat([mrna_train, cnv_train], axis=1)
test_data = pd.concat([mrna_test, cnv_test], axis=1)

train_columns = train_data.columns.tolist()
test_columns = test_data.columns.tolist()   

train_data.columns = [ i for i in range(len(train_columns))]
test_data.columns = [ i for i in range(len(test_columns))]

train_data.index = [ i for i in range(train_data.shape[0])]
test_data.index = [ i for i in range(test_data.shape[0])]
label_train.index = [ i for i in range(label_train.shape[0])]
label_test.index = [ i for i in range(label_test.shape[0])]

est , selected_gene , information_edge_tensor  = generate_ac_feature_selection(train_data.copy(deep=True) , label_train.copy(deep=True) , "" , n_bins=3 , df_test_data=test_data.copy(deep=True) , df_test_label=label_test.copy(deep=True) , fixed_k=1000)

Generate CARs for class 4.0: 100%|██████████| 5/5 [00:18<00:00,  3.80s/it]            


------ ARM summary [Gene Filter: 0]---------
    data_shape  itemset_length  support  support_percentage  cars_length  \
0  (123, 2000)            1564      115           93.495935         1564   
1   (56, 2000)            1092       44           78.571429         1092   
2  (389, 2000)            1259      338           86.889460         1259   
3  (153, 2000)            1033      133           86.928105         1033   
4   (25, 2000)            1111       19           76.000000         1111   

   avg_confidence  
0        0.667755  
1        0.268794  
2        0.542277  
3        0.204541  
4        0.286349  
Top 1000 | 0.5000 | Lens gene: 845


Generate information-edge tensor: 100%|██████████| 5000/5000 [01:24<00:00, 59.03it/s] 

Best K: 1000 | Best Acc: 0.5000


In [20]:
information_edge_tensor.shape

torch.Size([2000, 2000])

In [21]:
selected_gene = sorted(selected_gene)
selection = {0:0,1:0}
for gene in selected_gene: 
    if gene in range(0 , cnv_train.shape[1]):
        selection[0] += 1
    else:
        selection[1] += 1
print(f"Selected gene distribution: {selection}")

Selected gene distribution: {0: 280, 1: 565}


In [22]:
merged

,protein1_name,protein2_name,neighborhood,coexpression
0,ARF5,RALGPS2,0,45
1,ARF5,FHDC1,0,0
2,ARF5,ATP6V1E1,0,118
3,ARF5,CYTH2,0,56
4,ARF5,PSD3,0,0
...,...,...,...,...
13715399,RFX7,MPHOSPH9,0,60
13715400,RFX7,VCX,0,55
13715401,RFX7,YPEL2,0,0
13715402,RFX7,SAMD3,0,0


In [24]:
# generate ppi edge 
import torch
gene_df = pd.DataFrame({"gene_name":train_columns, "gene_idx": range(len(train_columns))})
merged = pd.merge(merged , gene_df, left_on="protein1_name", right_on="gene_name")
merged.rename(columns={"gene_idx":"gene1_idx"}, inplace=True)
merged = pd.merge(merged , gene_df, left_on="protein2_name", right_on="gene_name")
merged.rename(columns={"gene_idx":"gene2_idx"}, inplace=True)
ppi_tensor = torch.zeros((len(train_columns), len(train_columns)))
ppi_tensor[merged["gene1_idx"].values , merged['gene2_idx'].values] = torch.tensor(merged['neighborhood'] , dtype=torch.float)
ppi_tensor[merged["gene2_idx"].values , merged['gene1_idx'].values] = torch.tensor(merged['neighborhood'] , dtype=torch.float)


In [25]:
co_tensor = torch.zeros((len(train_columns), len(train_columns)))
co_tensor[merged["gene1_idx"].values , merged['gene2_idx'].values] = torch.tensor(merged['coexpression'] , dtype=torch.float)
co_tensor[merged["gene2_idx"].values , merged['gene1_idx'].values] = torch.tensor(merged['coexpression'] , dtype=torch.float)

In [26]:
ggi = GGI[["Official Symbol Interactor A", "Official Symbol Interactor B"]]
ggi.rename(columns={"Official Symbol Interactor A":"gene1_name", "Official Symbol Interactor B":"gene2_name"}, inplace=True)
ggi = pd.merge(ggi , gene_df, left_on="gene1_name", right_on="gene_name")
ggi.rename(columns={"gene_idx":"gene1_idx"}, inplace=True)
ggi = pd.merge(ggi , gene_df, left_on="gene2_name", right_on="gene_name")
ggi.rename(columns={"gene_idx":"gene2_idx"}, inplace=True)
ggi

,gene1_name,gene2_name,gene_name_x,gene1_idx,gene_name_y,gene2_idx
0,PFN2,FHOD1,PFN2,1002,FHOD1,1805
1,GATA1,ZFPM2,GATA1,132,ZFPM2,1255
2,AR,PAK6,AR,420,PAK6,1518
3,SYN2,SYN2,SYN2,979,SYN2,979
4,TRIM10,TRIM10,TRIM10,688,TRIM10,688
...,...,...,...,...,...,...
5322,KRT6A,KRT20,KRT6A,745,KRT20,38
5323,HP,KRT14,HP,1854,KRT14,161
5324,HP,KRT16,HP,1854,KRT16,197
5325,AMBP,ITIH2,AMBP,231,ITIH2,1427


In [27]:
ggi_tensor = torch.zeros((len(train_columns), len(train_columns)))
ggi_tensor[ggi["gene1_idx"].values , ggi['gene2_idx'].values] = 1
ggi_tensor[ggi["gene2_idx"].values , ggi['gene1_idx'].values] = 1

In [28]:
# print all information shape 
print(f"Train data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")
print(f"Train label shape: {label_train.shape}")
print(f"Test label shape: {label_test.shape}")
print(f"AC information shape: {information_edge_tensor.shape}")
print(f"PPI information shape: {ppi_tensor.shape}")
print(f"CO information shape: {co_tensor.shape}")
print(f"GGI information shape: {ggi_tensor.shape}")

Train data shape: (746, 2000)
Test data shape: (320, 2000)
Train label shape: (746, 1)
Test label shape: (320, 1)
AC information shape: torch.Size([2000, 2000])
PPI information shape: torch.Size([2000, 2000])
CO information shape: torch.Size([2000, 2000])
GGI information shape: torch.Size([2000, 2000])


In [31]:
type(selected_gene)

list

In [79]:
# create unified edges matrix 

general_edges = []
info_edge = information_edge_tensor[selected_gene][:,selected_gene]
info_edge[torch.isnan(info_edge)] = 0

mean_info_edge = info_edge.mean()

ppi_edge = ppi_tensor[selected_gene][: , selected_gene]*mean_info_edge
co_edge = co_tensor[selected_gene][: , selected_gene]*mean_info_edge
ggi_edge = ggi_tensor[selected_gene][: , selected_gene]*mean_info_edge

general_edges = torch.stack([info_edge , ppi_edge , co_edge , ggi_edge] , dim=-1)
print(f"General edges shape: {general_edges.shape}")

General edges shape: torch.Size([845, 845, 4])


In [80]:
general_edges[0][general_edges[0] > 0].count_nonzero()

tensor(0)

In [81]:
from tqdm import tqdm 
from torch.nn.functional import one_hot 
from amogel.utils.common import symmetric_matrix_to_pyg

train_transformed = pd.DataFrame(est.transform(train_data)).iloc[: , selected_gene]
train_graphs = []
with tqdm(total=train_data.shape[0] , desc="Generating training graphs") as pbar: 
    for idx, sample in train_transformed.iterrows():
        torch_sample = torch.tensor(sample.values , dtype=torch.float32).unsqueeze(-1)
        torch_sample = one_hot(torch_sample.long() , num_classes=3).squeeze(1).float() 
        graph = symmetric_matrix_to_pyg(
            matrix = general_edges ,
            node_features=torch_sample, 
            y=torch.tensor(label_train.loc[idx].values , dtype=torch.long), 
            edge_threshold=0.0
        )
        
        train_graphs.append(graph)
        pbar.update(1)
        
test_transformed = pd.DataFrame(est.transform(test_data)).iloc[: , selected_gene]
test_graphs = []
with tqdm(total=test_data.shape[0] , desc="Generating test graphs") as pbar: 
    for idx, sample in test_transformed.iterrows():
        torch_sample = torch.tensor(sample.values , dtype=torch.float32).unsqueeze(-1)
        torch_sample = one_hot(torch_sample.long() , num_classes=3).squeeze(1).float() 
        graph = symmetric_matrix_to_pyg(
            matrix = general_edges ,
            node_features=torch_sample, 
            y=torch.tensor(label_train.loc[idx].values , dtype=torch.long), 
            edge_threshold=0.0
        )
        
        test_graphs.append(graph)
        pbar.update(1)

Generating test graphs: 100%|██████████| 320/320 [00:12<00:00, 25.32it/s]


In [82]:
print(f"Node dimension: {test_graphs[0].x.shape} , Edge dimension: {test_graphs[0].edge_index.shape} , \
                    Edge attribute dimension: {test_graphs[0].edge_attr.shape} , \
                    Edge max: {test_graphs[0].edge_attr.max(dim=0).values} , \
                    Edge mean: {test_graphs[0].edge_attr.mean(dim=0)} , \
                    Nonzero edge: {torch.count_nonzero(test_graphs[0].edge_attr , dim=0)}")

Node dimension: torch.Size([845, 3]) , Edge dimension: torch.Size([2, 322390]) ,                     Edge attribute dimension: torch.Size([322390, 4]) ,                     Edge max: tensor([ 0.4034,  6.0181, 47.7151,  0.0478]) ,                     Edge mean: tensor([1.0578e-01, 2.9829e-03, 1.2105e-01, 1.1319e-04]) ,                     Nonzero edge: tensor([322390,    370,   7912,    764])
